# Scalable Data Science with Dask

Dataset: [NYC Yellow Taxi Trips [2019]](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

## pandas

Read the data for January 2019.

In [5]:
%%time

import pandas as pd

df = pd.read_csv("yellow_tripdata_2019-01.csv")
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.00,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.00,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.00,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7667787,2,2019-01-31 23:57:36,2019-02-01 00:18:39,1,4.79,1,N,263,4,1,18.0,0.5,0.5,3.86,0.0,0.3,23.16,0.0
7667788,2,2019-01-31 23:32:03,2019-01-31 23:33:11,1,0.00,1,N,193,193,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
7667789,2,2019-01-31 23:36:36,2019-01-31 23:36:40,1,0.00,1,N,264,264,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0
7667790,2,2019-01-31 23:14:53,2019-01-31 23:15:20,1,0.00,1,N,264,7,1,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0


Calculate mean of total_amount.

In [6]:
%%time

df.total_amount.mean()

CPU times: user 22 ms, sys: 13.7 ms, total: 35.7 ms
Wall time: 33.9 ms


15.68222215991253

## Dask

Start a cluster.

In [1]:
from dask.distributed import Client

client = Client(n_workers=4)
client

Client Scheduler: tcp://127.0.0.1:61120 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 12 Memory: 16.00 GiB


Read data for entire year 2019.

In [8]:
%%time

import dask.dataframe as dd

df = dd.read_csv("yellow_tripdata_2019-*.csv",
                 dtype={'RatecodeID': 'float64',
                        'VendorID': 'float64',
                        'passenger_count': 'float64',
                        'payment_type': 'float64'
                       })
df

CPU times: user 22.2 ms, sys: 33.5 ms, total: 55.6 ms
Wall time: 54.5 ms


Dask Dataframes are lazily evaluated, need to call `head()` to view elements.

In [12]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1.0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.5,1.0,N,151,239,1.0,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1.0,2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.6,1.0,N,239,246,1.0,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2.0,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.0,1.0,N,236,236,1.0,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2.0,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.0,1.0,N,193,193,2.0,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2.0,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.0,2.0,N,193,193,2.0,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


Calculate mean of total_amount.

In [13]:
%%time

df.total_amount.mean()

CPU times: user 1.85 ms, sys: 108 µs, total: 1.95 ms
Wall time: 1.96 ms


dd.Scalar<series-..., dtype=float64>

Again, lazy evaluation. Need to call `compute()` to compute result.

In [14]:
%%time

df.total_amount.mean().compute()

CPU times: user 6.72 s, sys: 1.36 s, total: 8.08 s
Wall time: 45.6 s


19.124363231638988

Close the cluster.

In [2]:
client.close()

## Coiled

Create a Dask cluster on Coiled.

In [3]:
from dask.distributed import Client

In [4]:
import dask.dataframe as dd

In [5]:
import coiled

cluster = coiled.Cluster(n_workers=10)

client = Client(cluster)
client

Output()

/opt/anaconda3/envs/pyladies-berlin-dask/lib/python3.8/site-packages/distributed/client.py:1140: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | client | scheduler | workers |
+---------+--------+-----------+---------+
| blosc   | None   | 1.10.2    | 1.10.2  |
| lz4     | None   | 3.1.3     | 3.1.3   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


Client Scheduler: tls://ec2-18-218-248-251.us-east-2.compute.amazonaws.com:8786 Dashboard: http://ec2-18-218-248-251.us-east-2.compute.amazonaws.com:8787,Cluster Workers: 10 Cores: 40 Memory: 160.00 GiB


Read data for January 2019 from Amazon S3 and compute the mean of total_amount.

In [6]:
%%time

df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-01.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
        "store_and_fwd_flag": "category",
        "PULocationID": "UInt16",
        "DOLocationID": "UInt16",
    },
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()

df.total_amount.mean().compute()

CPU times: user 420 ms, sys: 110 ms, total: 530 ms
Wall time: 18 s


15.682222159912529